In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("output.csv")

In [4]:
x = df.drop("num_orders", axis=1)
y = df['num_orders']

In [5]:
import evalml

In [6]:
evalml.problem_types.ProblemTypes.all_problem_types

[<ProblemTypes.BINARY: 'binary'>,
 <ProblemTypes.MULTICLASS: 'multiclass'>,
 <ProblemTypes.REGRESSION: 'regression'>,
 <ProblemTypes.TIME_SERIES_REGRESSION: 'time series regression'>,
 <ProblemTypes.TIME_SERIES_BINARY: 'time series binary'>,
 <ProblemTypes.TIME_SERIES_MULTICLASS: 'time series multiclass'>]

In [7]:
X_train,X_test,y_train,y_test=evalml.preprocessing.split_data(x,y,problem_type='time series regression')

In [8]:
from evalml import AutoMLSearch

In [11]:
automl=AutoMLSearch(X_train=X_train,y_train=y_train,problem_type='regression',max_batches=1,optimize_thresholds=True)

Generating pipelines to search over...


In [12]:
automl.search()


*****************************
* Beginning pipeline search *
*****************************

Optimizing for R2. 
Greater score is better.

Using SequentialEngine to train and score pipelines.
Searching up to 1 batches for a total of 9 pipelines. 
Allowed model families: lightgbm, xgboost, decision_tree, random_forest, catboost, linear_model, extra_trees



FigureWidget({
    'data': [{'mode': 'lines+markers',
              'name': 'Best Score',
              'type'…

Evaluating Baseline Pipeline: Mean Baseline Regression Pipeline
Mean Baseline Regression Pipeline:
	Starting cross validation
	Finished cross validation - mean R2: -0.000

*****************************
* Evaluating Batch Number 1 *
*****************************

Linear Regressor w/ Imputer + Standard Scaler:
	Starting cross validation
	Finished cross validation - mean R2: 0.665
Decision Tree Regressor w/ Imputer:
	Starting cross validation
	Finished cross validation - mean R2: 0.594
Random Forest Regressor w/ Imputer:
	Starting cross validation
	Finished cross validation - mean R2: 0.613
LightGBM Regressor w/ Imputer:
	Starting cross validation
	Finished cross validation - mean R2: 0.690
Elastic Net Regressor w/ Imputer + Standard Scaler:
	Starting cross validation
	Finished cross validation - mean R2: 0.167
XGBoost Regressor w/ Imputer:
	Starting cross validation
	Finished cross validation - mean R2: 0.800
Extra Trees Regressor w/ Imputer:
	Starting cross validation
	Finished cross va

In [13]:
automl.rankings

,id,pipeline_name,mean_cv_score,standard_deviation_cv_score,validation_score,percent_better_than_baseline,high_variance_cv,parameters
0,6,XGBoost Regressor w/ Imputer,0.799733,NaN,0.799733,2.738980e+06,False,{'Imputer': {'categorical_impute_strategy': 'm...
1,4,LightGBM Regressor w/ Imputer,0.689817,NaN,0.689817,2.362548e+06,False,{'Imputer': {'categorical_impute_strategy': 'm...
2,1,Linear Regressor w/ Imputer + Standard Scaler,0.665003,NaN,0.665003,2.277566e+06,False,{'Imputer': {'categorical_impute_strategy': 'm...
3,3,Random Forest Regressor w/ Imputer,0.613094,NaN,0.613094,2.099790e+06,False,{'Imputer': {'categorical_impute_strategy': 'm...
4,2,Decision Tree Regressor w/ Imputer,0.594243,NaN,0.594243,2.035230e+06,False,{'Imputer': {'categorical_impute_strategy': 'm...
5,7,Extra Trees Regressor w/ Imputer,0.592428,NaN,0.592428,2.029015e+06,False,{'Imputer': {'categorical_impute_strategy': 'm...
6,8,CatBoost Regressor w/ Imputer,0.235062,NaN,0.235062,8.051256e+05,False,{'Imputer': {'categorical_impute_strategy': 'm...
7,5,Elastic Net Regressor w/ Imputer + Standard Sc...,0.166930,NaN,0.166930,5.717919e+05,False,{'Imputer': {'categorical_impute_strategy': 'm...
8,0,Mean Baseline Regression Pipeline,-0.000029,NaN,-0.000029,0.000000e+00,False,{'Baseline Regressor': {'strategy': 'mean'}}


In [14]:
automl.best_pipeline

pipeline = RegressionPipeline(component_graph=[Imputer, XGBoostRegressor], parameters={'Imputer':{'categorical_impute_strategy': 'most_frequent', 'numeric_impute_strategy': 'mean', 'categorical_fill_value': None, 'numeric_fill_value': None}, 'XGBoost Regressor':{'eta': 0.1, 'max_depth': 6, 'min_child_weight': 1, 'n_estimators': 100}}, random_seed=0)

In [15]:
best_pipeline = automl.best_pipeline

In [16]:
automl.describe_pipeline(automl.rankings.iloc[0]["id"])


********************************
* XGBoost Regressor w/ Imputer *
********************************

Problem Type: regression
Model Family: XGBoost

Pipeline Steps
1. Imputer
	 * categorical_impute_strategy : most_frequent
	 * numeric_impute_strategy : mean
	 * categorical_fill_value : None
	 * numeric_fill_value : None
2. XGBoost Regressor
	 * eta : 0.1
	 * max_depth : 6
	 * min_child_weight : 1
	 * n_estimators : 100

Training
Training for regression problems.
Total training time (including CV): 191.7 seconds

Cross Validation
----------------
               R2 ExpVariance MaxError MedianAE   MSE   MAE Root Mean Squared Error # Training # Validation
0           0.800       0.800    3.627    0.349 0.300 0.427                   0.548     91,309      273,929
mean        0.800       0.800    3.627    0.349 0.300 0.427                   0.548          -            -
std             -           -        -        -     -     -                       -          -            -
coef of var     

In [17]:
scores = best_pipeline.score(X_test, y_test,  objectives=evalml.objectives.get_core_objectives('regression'))

In [18]:
print(scores)

OrderedDict([('ExpVariance', 0.7598364362794708), ('MaxError', 3.045752098397302), ('MedianAE', 0.38148178225077056), ('MSE', 0.34299842704365674), ('MAE', 0.4606519404955711), ('R2', 0.7595564536910264), ('Root Mean Squared Error', 0.5856606756848686)])


In [19]:
X_train.head()

Data Column,Unnamed: 0,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,city_code,region_code,...,center_type_TYPE_A,center_type_TYPE_B,center_type_TYPE_C,Quarter_Q1,Quarter_Q2,Quarter_Q3,Quarter_Q4,Year_Y1,Year_Y2,Year_Y3
Physical Type,Int64,Int64,Int64,Int64,float64,float64,Int64,Int64,Int64,Int64,...,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
Logical Type,Integer,Integer,Integer,Integer,Double,Double,Integer,Integer,Integer,Integer,...,Integer,Integer,Integer,Integer,Integer,Integer,Integer,Integer,Integer,Integer
Semantic Tag(s),['numeric'],['numeric'],['numeric'],['numeric'],['numeric'],['numeric'],['numeric'],['numeric'],['numeric'],['numeric'],...,['numeric'],['numeric'],['numeric'],['numeric'],['numeric'],['numeric'],['numeric'],['numeric'],['numeric'],['numeric']
0,0,1,55,1885,136.83,152.29,0,0,647,56,...,0,0,1,1,0,0,0,1,0,0
1,1,1,55,1993,136.83,135.83,0,0,647,56,...,0,0,1,1,0,0,0,1,0,0
2,2,1,55,2539,134.86,135.86,0,0,647,56,...,0,0,1,1,0,0,0,1,0,0
3,3,1,55,2139,339.50,437.53,0,0,647,56,...,0,0,1,1,0,0,0,1,0,0
4,4,1,55,2631,243.50,242.50,0,0,647,56,...,0,0,1,1,0,0,0,1,0,0
